In [26]:
from dateutil.relativedelta import relativedelta
import swifter
import pandas as pd

## Standard Features
### {Fighter A, Fighter B} x {significant strikes landed per min (s/m), significant striking accuracy (%), significant strikes absorbed per minute (s/m), significant strike defense (s/m)} x {R1, R2, R3, R4, R5, overall} x {last 3 fights, last 5 fights, alltime}


## Differential Features
### {Fighter A, Fighter B} x {significant strikes differential, significant strikes accuracy differential, significant strikes absorbed differential, significant strikes defended differential} x {R1, R2, R3, R4, R5, overall} x {last 3 fights, last 5 fights, alltime}

In [27]:
def create_col_names_significant_strikes():
        # private
        col_names = []
        fighters = ['fighter-a', 'fighter-b']
        significant_strike_stats = ['significant-strikes-landed', 'significant-strikes-accuracy', 'significant-strikes-defense', 'significant-strikes-absorbed']
        rounds = ['R1', 'R2', 'R3', 'R4', 'R5', 'overall']
        time_periods = ['last-3-fights', 'last-5-fights', 'alltime']

        for fighter in fighters:
            for significant_strike_stat in significant_strike_stats:
                for round in rounds:
                    for time_period in time_periods:
                        col_name = f"{fighter.replace('-', '_')}_{significant_strike_stat.replace('-', '_')}_{round}_{time_period.replace('-', '_')}"
                        col_names.append(col_name)
        return col_names

def create_col_names_differential_significant_strikes():
        # private
        col_names = []
        fighters = ['fighter-a', 'fighter-b']
        significant_strike_stats = ['significant-strikes-differential', 'significant-strikes-accuracy-differential', 'significant-strikes-defense-differential', 'significant-strikes-absorbed-differential']
        rounds = ['R1', 'R2', 'R3', 'R4', 'R5', 'overall']
        time_periods = ['last-3-fights', 'last-5-fights', 'alltime']

        for fighter in fighters:
            for significant_strike_stat in significant_strike_stats:
                for round in rounds:
                    for time_period in time_periods:
                        col_name = f"{fighter.replace('-', '_')}_{significant_strike_stat.replace('-', '_')}_{round}_{time_period.replace('-', '_')}"
                        col_names.append(col_name)
        return col_names






In [28]:
col_names = create_col_names_significant_strikes()
col_names_differential = create_col_names_differential_significant_strikes()

print(len(col_names) + len(col_names_differential))

288


In [41]:
def get_fighter_fights_significant_strikes(df, fighter_id, index):
    all_prev_fights = df.loc[:index-1]

    if not all_prev_fights.empty:
        fighter_a_id_vals = all_prev_fights.fighter_a_id.values
        fighter_b_id_vals = all_prev_fights.fighter_b_id.values

        prev_fights = all_prev_fights[((fighter_a_id_vals == fighter_id) | (fighter_b_id_vals == fighter_id))]
        num_fights = len(prev_fights)

        # Initialize empty DataFrames for the scenarios where there are fewer than 3 or 5 fights
        last_3_fights = pd.DataFrame()
        last_5_fights = pd.DataFrame()

        # Check if the fighter has at least 3 fights
        if num_fights >= 3:
            last_3_fights = prev_fights.tail(3)
        else:
            # If fewer than 3 fights, return all available fights
            last_3_fights = prev_fights

        # Check if the fighter has at least 5 fights
        if num_fights >= 5:
            last_5_fights = prev_fights.tail(5)
        else:
            # If fewer than 5 fights, return all available fights
            last_5_fights = prev_fights

        return last_3_fights, last_5_fights, prev_fights

    return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()


def get_raw_significant_strikes_stats(df, round):
    retVal = None # somthing to return the correct stats based on round 
    # Ex: 
    # [fighter_a_round_1_sig_str_landed', 'fighter_a_round_1_sig_str_attempted', 'fighter_a_round_1_sig_str_pct', 'fighter_a_round_1_total_str_landed', 'fighter_a_round_1_total_str_attempted']

    return retVal

# [fighter_a_round_1_sig_str_landed', 'fighter_a_round_1_sig_str_attempted', 'fighter_a_round_1_sig_str_pct', 'fighter_a_round_1_total_str_landed', 'fighter_a_round_1_total_str_attempted']

def calculate_significant_strikes_landed(stats):
    for i in range(len(stats)):
        temp_significant_strikes_landed += stats[i][0]

    temp_significant_strikes_landed = temp_significant_strikes_landed/len(stats)
    return None

def calculate_significant_strikes_accuracy(stats):
    for i in range(len(stats)):
        temp_significant_strikes_accuracy += (stats[i][0]/stats[i][1])
    
    temp_significant_strikes_accuracy = temp_significant_strikes_accuracy/len(stats)

    return temp_significant_strikes_accuracy


def calculate_significant_strikes_defense(stats):
    for i in range(len(stats)):
        
        
        pass
    return None


def calculate_significant_strikes_absorbed(stats):
    for i in range(len(stats)):
        pass
    return None

In [37]:
def calculate_significant_strikes(df, fighter_a_id, fighter_b_id, index, col_names):
    # private
    res = []
    fighter_a_prev_fights, fighter_a_last_3_fights, fighter_a_last_5_fights = get_fighter_fights_significant_strikes(df, fighter_a_id, index)
    fighter_b_prev_fights, fighter_b_last_3_fights, fighter_b_last_5_fights = get_fighter_fights_significant_strikes(df, fighter_b_id, index)

    for i in range(0, len(col_names)):
        split = col_names[i].split('_')
        fighter = split[0]
        stat = split[1]
        round = split[2][1]
        time_period = split[3]

        print(fighter, stat, round, time_period)

        # Get fights for the fighter
        all_prev_fights = fighter_a_prev_fights if fighter == 'fighter-a' else fighter_b_prev_fights
        last_3_fights = fighter_a_last_3_fights if fighter == 'fighter-a' else fighter_b_last_3_fights
        last_5_fights = fighter_a_last_5_fights if fighter == 'fighter-a' else fighter_b_last_5_fights
        
        prev_fights = None


        # filter based on time period
        if time_period == 'last_3_fights':
            prev_fights = last_3_fights

        if time_period == 'last_5_fights':
            prev_fights = last_5_fights

        if time_period == 'alltime':
            prev_fights = all_prev_fights
        

        # if no fights 
        if prev_fights.empty:
                res.append(0) #### CHANGE DEFAULT VAL MAYBE
                continue
        
        # filter and get the stats for the specific round
        raw_significant_strike_stats = get_raw_significant_strikes_stats(prev_fights, round)

        if raw_significant_strike_stats.empty:
             res.append() #### CHANGE DEFAULT VAL MAYBE
             continue

        # filter for the needed stat
        if stat == " significant_strikes":
             res.append(calculate_significant_strikes_landed(raw_significant_strike_stats))
        if stat == " significant_strikes_accuracy":
             res.append(calculate_significant_strikes_accuracy(raw_significant_strike_stats))
        if stat == " significant_strikes_defense":
             res.append(calculate_significant_strikes_defense(raw_significant_strike_stats))
        if stat == " significant_strikes_absorbed":
             res.append(calculate_significant_strikes_absorbed(raw_significant_strike_stats))



        # fighters = ['fighter-a', 'fighter-b']
        # significant_strike_stats = ['significant-strikes-landed', 'significant-strikes-accuracy', 'significant-strikes-defense', 'significant-strikes-absorbed']
        # rounds = ['R1', 'R2', 'R3', 'R4', 'R5', 'overall']
        # time_periods = ['last-3-fights', 'last-5-fights', 'alltime']
        # [fighter_a_round_1_sig_str_landed', 'fighter_a_round_1_sig_str_attempted', 'fighter_a_round_1_sig_str_pct', 'fighter_a_round_1_total_str_landed', 'fighter_a_round_1_total_str_attempted']

    return None

In [35]:
def calculate_significant_strikes_differential(df, fighter_a_id, fighter_b_id, index, col_names):
    # private
    return None

In [31]:
def create_win_loss_feats(input_df):
        col_names = create_col_names_significant_strikes()
        col_names_differential = create_col_names_differential_significant_strikes()
       

        input_df[col_names] = input_df.swifter.progress_bar(True).apply(lambda row: calculate_significant_strikes(input_df, row['fighter_a_id'], row['fighter_b_id'], row.name, col_names), axis=1)
        input_df[col_names_differential] = input_df.swifter.progress_bar(True).apply(lambda row: calculate_significant_strikes_differential(input_df, row['fighter_a_id'], row['fighter_b_id'], row.name, col_names_differential), axis=1)
        return input_df

In [43]:
x = [1, 2, 3, 4]

for i in range(len(x)):
    print(len(x))

4
4
4
4
